In [1]:
%reload_ext autoreload
%autoreload 2

import os
import argparse
import sys
import time
import cv2

from joblib import Parallel, delayed

sys.path.append(os.path.join(os.environ['REPO_DIR'], 'utilities'))
from utilities2015 import *

from matplotlib.path import Path
%matplotlib inline

import scipy.ndimage as nd
import scipy

from sklearn.cluster import KMeans

import random

In [2]:
stack = 'MD585'

In [3]:
dm = DataManager(stack=stack)

In [33]:
xmin, ymin, w, h = detect_bbox_lookup[stack]
xmin = xmin * 32
ymin = ymin * 32
w = w * 32
h = h * 32
xmax = xmin + w - 1
ymax = ymin + h - 1

In [32]:
first_detect_sec, last_detect_sec = detect_bbox_range_lookup[stack]

In [ ]:
for sec in range(first_detect_sec, last_detect_sec+1):
    dm.set_slice(sec)
    dm._load_image(versions=['rgb-jpg'])
    dm.image_rgb_jpg[ymin:ymax+1, xmin:xmax+1]

In [36]:
sec = 161
dm.set_slice(sec)
dm._load_image(versions=['rgb-jpg'])

In [40]:
img = dm.image_rgb_jpg[ymin:ymax+1, xmin:xmax+1]

In [38]:
sift = cv2.SIFT();

In [ ]:
img.shape

In [ ]:
t = time.time()
key_point, descriptor = sift.detectAndCompute(img[:100], None);
sys.stderr.write('load scoremap: %.2f seconds\n' % (time.time() - t))

In [ ]:
# Return the histogram of an image by applying SIFT + K-means + SPM
# image: the input grayscaled image
# kmeans: python kmeans object used to predict
#             feature type of a new sift descriptor
# M: number of channels (feature typs)
# L: number of SPM levels.
def getHistogram(image, kmeans, M, L):
    x_size = image.shape[0];
    y_size = image.shape[1];
    # Get SIFT key points and descriptor.
    sift = cv2.SIFT();
    key_point, descriptor = sift.detectAndCompute(image, None);
    # Map key points with type.
    cluster_labels = kmeans.predict(descriptor);
    
    # Level 0
    histogram = np.zeros(M);
    weight_0 = 1.0 / (2**L);
    for k in range(0, len(key_point)):
        histogram[cluster_labels[k]] += 1;
    histogram = histogram * weight_0;
    # Other levels
    for l in range(1, L):
        grid_x = 1.0 * x_size / (2**l);
        grid_y = 1.0 * y_size / (2**l);
        weight = 1.0 / (2**(L - l + 1));
        for i in range(0, (2**l)):
            x_min = i * grid_x;
            x_max = x_min + grid_x;
            for j in range(0, (2**l)):
                y_min = j * grid_y;
                y_max = y_min + grid_y;
                histogram_tmp = np.zeros(M);
                # Count and update histogram from this grid.
                for k in range(0, len(key_point)):
                    if (x_min <= key_point[k].pt[0] <= x_max and
                        y_min <= key_point[k].pt[1] <= y_max):
                        histogram_tmp[cluster_labels[k]] += 1;
                histogram_tmp = histogram_tmp * weight;
                histogram = np.hstack((histogram, histogram_tmp));
    return (histogram / np.sum(histogram)); # Normalization